# Imports  
  
https://github.com/WillKoehrsen/wikipedia-data-science/blob/master/notebooks/Downloading%20and%20Parsing%20Wikipedia%20Articles.ipynb

In [31]:
import requests
from bs4 import BeautifulSoup
import keras as tf
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()
from tensorflow.keras.utils import get_file
#from keras.utils import data_utils
#from keras.utils import get_file

Segmentation Models: using `keras` framework.


In [54]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [56]:
import bz2
import subprocess

In [33]:
import tensorflow
print(tensorflow)

<module 'tensorflow' from 'C:\\Users\\ninap\\miniconda3\\envs\\dmml\\lib\\site-packages\\tensorflow\\__init__.py'>


# Get Dumps

In [34]:
base_url = 'https://dumps.wikimedia.org/enwiki/'
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, 'html.parser')

In [ ]:
dumps = [a['href'] for a in soup_index.find_all('a') if a.has_attr('href')]
dumps

In [36]:
dump_url = base_url + '20220201/'

In [37]:
dump_html = requests.get(dump_url).text

In [38]:
soup_dump = BeautifulSoup(dump_html, 'html.parser')

In [ ]:
soup_dump.find_all('li', {'class': 'file'})[:3]

In [ ]:
files = []

# Search through all files
for file in soup_dump.find_all('li', {'class': 'file'}):
    text = file.text
    # Select the relevant files
    if 'meta-history1' in text:
        files.append((text.split()[0], text.split()[1:]))
        
files[0:-5]



In [139]:
files.clear()

In [ ]:
files_to_download = [file[0] for file in files if '.xml-p' and '.bz2' in file[0]]
files_to_download[0:20]

In [ ]:
#files_to_download.clear()

In [85]:
keras_home = 'C:/Users/ninap/.keras/datasets/'

In [ ]:
data_paths = []
file_info = []

# Iterate through each file
for file in files_to_download:
    path = keras_home + file
    
    # Check to see if the path exists (if the file is already downloaded)
    if not os.path.exists(keras_home + file):
        print('Downloading')
        # If not, download the file
        data_paths.append(get_file(fname=file, origin = dump_url+file))
        # Find the file size in MB
        file_size = os.stat(path).st_size / 1e6
        
        # Find the number of articles
        file_articles = int(file.split('p')[-1].split('.')[-2]) - int(file.split('p')[-2])
        file_info.append((file, file_size, file_articles))
        
    # If the file is already downloaded find some information
    else:
        data_paths.append(path)
        # Find the file size in MB
        file_size = os.stat(path).st_size / 1e6
        
        # Find the number of articles
        file_number = int(file.split('p')[-1].split('.')[-2]) - int(file.split('p')[-2])
        file_info.append((file.split('-')[-1], file_size, file_number))

# Check Files

In [ ]:
data_paths

In [145]:
data_paths.clear()
file_info.clear()

In [ ]:
sorted(file_info, key = lambda x: x[1], reverse = True)[:5]

In [ ]:
sorted(file_info, key = lambda x: x[2], reverse = True)[:5]

In [ ]:
print(f'There are {len(file_info)} partitions.')

In [ ]:
file_df = pd.DataFrame(file_info, columns = ['file', 'size (MB)', 'articles']).set_index('file')
file_df['size (MB)'].plot.bar(color = 'red', figsize = (12, 6));

In [ ]:
data_path = data_paths[15]
data_path

In [ ]:
lines = []

for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    lines.append(line)
    if i > 5e5:
        break
        
lines[-165:-109]